# Cortisol Analysis

In [ ]:
from pathlib import Path
import json

import pandas as pd
import numpy as np
import pingouin as pg

import biopsykit as bp
from biopsykit.stats import StatsPipeline
from biopsykit.io import load_long_format_csv
from biopsykit.utils.dataframe_handling import multi_xs

from fau_colors import cmaps, register_fausans_font

import matplotlib.pyplot as plt
import seaborn as sns

from carwatch_analysis.io import load_cortisol_samples_reporting_times
from carwatch_analysis.utils import describe_groups_df
from carwatch_analysis.stats import create_unique_night_id
from carwatch_analysis.plotting import multi_paired_plot_auc, paired_plot_auc
from carwatch_analysis.data_processing.saliva import compute_auc_increasing


%matplotlib widget
%load_ext autoreload
%autoreload 2

In [ ]:
register_fausans_font()
plt.close("all")

palette = sns.color_palette(cmaps.faculties)

theme_kwargs = {"context": "talk", "style": "ticks", "palette": palette}
theme_kwargs_grid = {"context": "talk", "style": "ticks", "palette": palette, "font_scale": 0.8}
sns.set_theme(**theme_kwargs)

plt.rcParams["figure.figsize"] = (10, 5)
plt.rcParams["pdf.fonttype"] = 42
plt.rcParams["mathtext.default"] = "regular"
plt.rcParams["font.family"] = "sans-serif"
plt.rcParams["font.sans-serif"] = "FAUSans Office"

export = True

pg.options["round"] = 4

palette

## Setup Paths

In [ ]:
base_path = Path("../..")
export_path = base_path.joinpath("exports")

## Load Data

### Cortisol Samples

In [ ]:
cort_path = export_path.joinpath("cortisol_samples_processed_all_reporting_types.csv")
cort_samples = load_cortisol_samples_reporting_times(cort_path)

cort_samples.head()

### Cortisol Features

In [ ]:
cort_features = load_long_format_csv(export_path.joinpath("cortisol_features_processed_all_reporting_types.csv"))
cort_features = create_unique_night_id(cort_features)
cort_features.head()

In [ ]:
# Don't consider IMU and IMU_App because the cortisol features are the same as
# Selfreport and App since only the wake onset differs, but not the sampling times
reporting_types_fine = [
    "Naive",
    "AW & ST: Selfreport (without App)",
    "AW & ST: Selfreport (with App)",
    "AW & ST: App",
    "AW: Sensor, ST: Selfreport (without App)",
    "AW: Sensor, ST: Selfreport (with App)",
    "AW: Sensor, ST: App",
]
reporting_types_coarse = [
    "Naive",
    "AW & ST: Selfreport",
    "AW & ST: App",
    "AW: Sensor, ST: Selfreport",
    "AW: Sensor, ST: App",
]
delay_groups = ["None", "Short", "Moderate", "High"]

reporting_types_fine_rename = [s.replace("without", "w/o").replace("with", "w/") for s in reporting_types_fine]
rename_mapper_reporting_types = dict(zip(reporting_types_fine, reporting_types_fine_rename))

reporting_types_fine = reporting_types_fine_rename

cort_samples = cort_samples.rename(rename_mapper_reporting_types, level="reporting_type")
cort_features = cort_features.rename(rename_mapper_reporting_types, level="reporting_type")

## Data Selection

### Cortisol Samples

In [ ]:
cort_samples = multi_xs(cort_samples, reporting_types_fine, level="reporting_type")
cort_samples.head()

### Cortisol Features

In [ ]:
cort_features_analysis = cort_features.copy()
cort_features_analysis = cort_features_analysis.reindex(["auc_g", "auc_i"], level="saliva_feature")
cort_features_analysis = cort_features_analysis.reindex(reporting_types_fine, level="reporting_type")
cort_features_analysis.head()

## Saliva Samples – CAR Plot

In [ ]:
reporting_types = ["AW & ST: Selfreport (w/o App)", "AW & ST: App"]

data_plot = cort_samples.reindex(reporting_types, level="reporting_type")

car = bp.protocols.CAR()
car.add_saliva_data(saliva_data=data_plot, saliva_type="cortisol", sample_times=[0, 15, 30, 45, 60])

In [ ]:
fig, ax = plt.subplots()

car.car_saliva_plot(
    saliva_type="cortisol",
    hue="reporting_type",
    style="reporting_type",
    ax=ax,
    hue_order=reporting_types,
    x_offset=0.0,
)
fig.tight_layout()

## Saliva Features – Statistical Analysis

### Log Type

In [ ]:
reporting_types = ["Naive", "AW & ST: Selfreport (w/ App)", "AW & ST: App"]

data_analysis = cort_features_analysis.reindex(reporting_types, level="reporting_type")

pipeline = StatsPipeline(
    steps=[("prep", "normality"), ("prep", "equal_var"), ("test", "rm_anova"), ("posthoc", "pairwise_tests")],
    params={
        "dv": "cortisol",
        "within": "reporting_type",
        "subject": "night_id",
        "groupby": "saliva_feature",
        "multicomp": {"method": "bonf", "levels": False},
    },
)

pipeline.apply(data_analysis)
pipeline.display_results()

#### Both AUCs

In [ ]:
title_map = {"auc_g": "$AUC_G$", "auc_i": "$AUC_I$"}

fig, axs = plt.subplots(figsize=(12, 5), ncols=2)

for (feature, data), ax in zip(cort_features_analysis.groupby("saliva_feature"), axs):
    bp.protocols.plotting.saliva_feature_boxplot(
        data=data,
        x="reporting_type",
        saliva_type="cortisol",
        feature=feature,
        order=reporting_types,
        palette=cmaps.faculties_light,
        ax=ax,
    )
    ax.set_title(title_map[feature], pad=12)
    ax.set_xlabel("Log Type")
    ax.set_ylim(ax.get_ylim()[0], ax.get_ylim()[1] * 1.05)

fig.tight_layout()

### Interaction Condition x Log Type

In [ ]:
reporting_types = ["Naive", "AW & ST: Selfreport (w/ App)", "AW & ST: App"]

data_analysis = cort_features_analysis.reindex(reporting_types, level="reporting_type")

pipeline = StatsPipeline(
    steps=[("prep", "normality"), ("prep", "equal_var"), ("test", "mixed_anova"), ("posthoc", "pairwise_tests")],
    params={
        "dv": "cortisol",
        "within": "reporting_type",
        "between": "condition",
        "subject": "night_id",
        "groupby": "saliva_feature",
        "multicomp": {"method": "bonf", "levels": None},
    },
)

pipeline.apply(data_analysis)
pipeline.display_results(prep=False, posthoc=False)

### Paired Plots

#### $AUC_G$

In [ ]:
data_increase = compute_auc_increasing(cort_features_analysis.reindex(reporting_types, level="reporting_type"), "auc_g")
data_increase

In [ ]:
fig, ax = paired_plot_auc(
    data=cort_features_analysis, saliva_feature="auc_g", reporting_types=reporting_types, figsize=(6, 5)
)

In [ ]:
reporting_types = ["Naive", "AW & ST: Selfreport (w/ App)", "AW & ST: App"]

fig, axs = multi_paired_plot_auc(
    data=cort_features_analysis, saliva_feature="auc_g", reporting_types=reporting_types, figsize=(12, 5)
)

#### $AUC_I$

In [ ]:
data_increase = compute_auc_increasing(cort_features_analysis.reindex(reporting_types, level="reporting_type"), "auc_i")
data_increase

In [ ]:
fig, ax = paired_plot_auc(
    data=cort_features_analysis, saliva_feature="auc_i", reporting_types=reporting_types, figsize=(6, 5)
)

In [ ]:
reporting_types = ["Naive", "AW & ST: Selfreport (w/ App)", "AW & ST: App"]

fig, axs = multi_paired_plot_auc(
    data=cort_features_analysis, saliva_feature="auc_i", reporting_types=reporting_types, figsize=(12, 5)
)

### Increase vs. S3 Time Delay

(Kudielka et al. 2003):  
"We found that the larger the time deviation for sample 3 (+30 min), the smaller the observed awakening cortisol increase. If subjects delay sample 3, they obviously miss the peak, and the resulting awakening increase turns out to be smaller."

#### Data Preparation

Get time deviation for S3 (+30 min)

In [ ]:
s3_delay = cort_samples.xs("S3", level="sample")[["time_diff_to_naive_min"]].dropna()

# drop time deviation outlier, i.e., samples that are be closer to S3 or to S1 than S2 (|delay| >= 7.5 min)
# drop_mask = s2_delay["time_diff_to_naive_min"].abs() >= 7.5
# drop_mask = drop_mask[drop_mask]
# s2_delay = s2_delay.drop(drop_mask.index)
s3_delay.columns = ["s3_delay"]

s3_delay.head()

Get cortisol increase between S1 and S3

In [ ]:
cort_inc = cort_samples[["cortisol"]]
cort_inc = cort_inc.xs("S3", level=-1) - cort_inc.xs("S1", level=-1)
cort_inc = cort_inc.join(s3_delay).dropna()
cort_inc.head()

#### Linear Regression

In [ ]:
data_grp = cort_inc.groupby("reporting_type")

data_result = {}

for reporting_type in ["AW & ST: Selfreport (w/o App)", "AW & ST: App", "AW: Sensor, ST: App"]:
    data_reg = data_grp.get_group(reporting_type)
    reg = pg.regression.linear_regression(
        X=data_reg["s3_delay"],
        y=data_reg["cortisol"],
    )
    data_result[reporting_type] = reg

pd.concat(data_result)

#### Regression Plot

In [ ]:
fig, ax = plt.subplots()

for reporting_type in ["AW & ST: Selfreport (w/o App)", "AW & ST: App", "AW: Sensor, ST: App"]:
    sns.regplot(
        data=cort_inc.xs(reporting_type, level="reporting_type").reset_index(),
        x="s3_delay",
        y="cortisol",
        ax=ax,
        label=reporting_type,
    )
ax.legend()
fig.tight_layout()

## Export